## Reads in price data, creates some variables, and saves it as into a monthly and annual CSV

In [19]:
import pandas as pd
import datetime
import numpy as np
import statsmodels.formula.api as sm
import statsmodels.api as smap
import matplotlib.pyplot as plt
# pd.set_option('display.max_rows', None)

In [30]:
outDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/StataDatasets/RawCSV/'

## Read in the raw excel data

In [20]:
data = pd.ExcelFile('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.xlsx')

df = data.parse(sheetname=data.sheet_names[0], skiprows=0)

## Clean up the column names since they are multi-indexed

In [21]:
# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x).lower() for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x).lower() for x in cols2]

cols3 = list(df.iloc[1,:])
cols3 = [str(x).lower().replace(" ","_") for x in cols3]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]
name = ""
for i,x in enumerate(cols):
    if "unnamed" in x:
        x = x.split('_')[-1]
        cols[i] = x
    if "_" in x:
        name = x.split("_")[0]
        continue
    elif ("_" not in x):
        cols[i] = name+"_" + x
cols[0:4] = cols3[0:4]
# Assign new column names to the dataframe
df.columns = cols
df.drop([0,1], inplace=True)
df = df[:-1]

## Standardize dates

In [22]:
df['date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(df.year, df.month)])

### Drop DC because it's a mess, and drop observations > 2008 to reduce massive data size

In [23]:
df = df[df['state']!="DC"]
df = df[df['year']>2008]

## Calculate the % monthly revenue from different sources

In [25]:
for x in ['residential', 'commercial', 'industrial']:
    df['perc_' + x + '_rev'] = pd.to_numeric(df[x+'_revenue'])/pd.to_numeric(df['total_revenue'])

### Save only a subset of columns to reduce data size

In [27]:
colList = ['year','month','state','residential_price','perc_residential_rev','perc_commercial_rev','perc_industrial_rev']
price_data = df[colList]

## Save in RawCSV Stata Data folder

In [31]:
price_data.to_csv(outDir + 'monthly_prices_by_state.csv')